In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir("../input/test/images"))

# Импорт библиотек

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import random
import os
import glob

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from keras.models import Model, load_model
from keras_preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers.core import Lambda, RepeatVector, Reshape
from segmentation_models import *
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

from tensorflow.python.keras.applications import vgg16

from tqdm import tqdm
import numpy as np
import pandas as pd

from random import randint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from keras.losses import binary_crossentropy
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation

In [ ]:
# посмотрим на данные

In [ ]:
img_size = 101
img_size_resize = 128

In [ ]:
train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
depth_df = pd.read_csv('../input/depths.csv')

In [ ]:
depth_df.info()

In [ ]:
test_df.info()

In [ ]:
#загружаю изображения и маски в датафрейм и нормализую

In [ ]:
train_df["images"] = [np.array(load_img("../input/train/images/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
train_df["masks"] = [np.array(load_img("../input/train/masks/{}.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_df.index)]

In [ ]:
#Расчет солевого покрытия и классов солевого покрытия Подсчет количества соляных пикселей в масках и деление их на размер изображения. Также создайте 11 классов покрытия, от -0.1 без соли до 1.0 только от соли. Построение графика распределения покрытий и классов покрытия, а также класса по необработанному покрытию.

In [ ]:
train_df["coverage"] = train_df.masks.map(np.sum) / pow(img_size, 2)

In [ ]:
#создаем классы по площади соляного покрытия на картинке

In [ ]:
def coverage_to_class(value):    
    for i in range(0, 11):
        if value * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df['coverage'].map(coverage_to_class)

In [ ]:
# взглянем на гистограммы(площадь покрытия и классы покрытия)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,5))
sns.distplot(train_df.coverage, kde=False, ax=axs[0])
sns.distplot(train_df.coverage_class, bins=10, kde=False, ax=axs[1])
plt.suptitle("Salt coverage")
axs[0].set_xlabel("Coverage")
axs[1].set_xlabel("Coverage class")

In [ ]:
# посмотрим на распределение глубины в тестовых и обучающих данных

In [ ]:
sns.distplot(train_df.z, label="Train")
sns.distplot(test_df.z, label="Test")
plt.legend()
plt.title("Depth distribution")

In [ ]:
# посмотрим на некотрые изображения

In [ ]:
max_images = 30
grid_width = 15
grid_height = int(max_images / grid_width)
fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width, grid_height))
for i, idx in enumerate(train_df.index[:max_images]):
    img = train_df.loc[idx].images
    mask = train_df.loc[idx].masks
    ax = axs[int(i / grid_width), i % grid_width]
    ax.imshow(img, cmap="Greys")
    ax.imshow(mask, alpha=0.4, cmap="Blues")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
plt.suptitle("Blue: salt")

In [ ]:
#разделяю данные на обучающую и валидационную выборку с учетом созданного признака:покрытие солью

In [ ]:
import sys

In [ ]:
im_height, im_width, im_chan = 128, 128, 1

In [ ]:
path_train = '../input/train/'
path_test = '../input/test/'

In [ ]:
train_ids = next(os.walk(path_train+"images"))[2]
test_ids = next(os.walk(path_test+"images"))[2]

In [ ]:
X_ = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.uint8)
Y_ = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    img = load_img(path + '/images/' + id_)
    x = img_to_array(img)[:,:,1]
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    X_[n] = x
    mask = img_to_array(load_img(path + '/masks/' + id_))[:,:,1]
    Y_[n] = resize(mask, (128, 128, 1), mode='constant', preserve_range=True)

In [ ]:
X_.shape

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_,Y_, random_state=20, test_size = 0.2)

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
#Построим простую модель 

In [ ]:
ix = random.randint(0, len(X_train))
has_mask = Y_train[ix].max() > 0 

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 15))

ax1.imshow(X_train[ix, ..., 0], cmap = 'Greys', interpolation = 'bilinear')
if has_mask: 
    ax1.contour(Y_train[ix].squeeze(), colors = 'k', linewidths = 5, levels = [0.5])
ax1.set_title('Seismic')

ax2.imshow(Y_train[ix].squeeze(), cmap = 'gray', interpolation = 'bilinear')
ax2.set_title('Salt')

In [ ]:
inp = Input(shape=(128, 128, 1))

conv_1_1 = Conv2D(32, (3, 3), padding='same')(inp)
conv_1_1 = Activation('relu')(conv_1_1)

conv_1_2 = Conv2D(32, (3, 3), padding='same')(conv_1_1)
conv_1_2 = Activation('relu')(conv_1_2)

pool_1 = MaxPooling2D(2)(conv_1_2)


conv_2_1 = Conv2D(64, (3, 3), padding='same')(pool_1)
conv_2_1 = Activation('relu')(conv_2_1)

conv_2_2 = Conv2D(64, (3, 3), padding='same')(conv_2_1)
conv_2_2 = Activation('relu')(conv_2_2)

pool_2 = MaxPooling2D(2)(conv_2_2)


conv_3_1 = Conv2D(128, (3, 3), padding='same')(pool_2)
conv_3_1 = Activation('relu')(conv_3_1)

conv_3_2 = Conv2D(128, (3, 3), padding='same')(conv_3_1)
conv_3_2 = Activation('relu')(conv_3_2)

pool_3 = MaxPooling2D(2)(conv_3_2)


conv_4_1 = Conv2D(256, (3, 3), padding='same')(pool_3)
conv_4_1 = Activation('relu')(conv_4_1)

conv_4_2 = Conv2D(256, (3, 3), padding='same')(conv_4_1)
conv_4_2 = Activation('relu')(conv_4_2)

pool_4 = MaxPooling2D(2)(conv_4_2)

up_1 = UpSampling2D(2, interpolation='bilinear')(pool_4)
conc_1 = Concatenate()([conv_4_2, up_1])

conv_up_1_1 = Conv2D(256, (3, 3), padding='same')(conc_1)
conv_up_1_1 = Activation('relu')(conv_up_1_1)

conv_up_1_2 = Conv2D(256, (3, 3), padding='same')(conv_up_1_1)
conv_up_1_2 = Activation('relu')(conv_up_1_2)


up_2 = UpSampling2D(2, interpolation='bilinear')(conv_up_1_2)
conc_2 = Concatenate()([conv_3_2, up_2])

conv_up_2_1 = Conv2D(128, (3, 3), padding='same')(conc_2)
conv_up_2_1 = Activation('relu')(conv_up_2_1)

conv_up_2_2 = Conv2D(128, (3, 3), padding='same')(conv_up_2_1)
conv_up_2_2 = Activation('relu')(conv_up_2_2)


up_3 = UpSampling2D(2, interpolation='bilinear')(conv_up_2_2)
conc_3 = Concatenate()([conv_2_2, up_3])

conv_up_3_1 = Conv2D(64, (3, 3), padding='same')(conc_3)
conv_up_3_1 = Activation('relu')(conv_up_3_1)

conv_up_3_2 = Conv2D(64, (3, 3), padding='same')(conv_up_3_1)
conv_up_3_2 = Activation('relu')(conv_up_3_2)



up_4 = UpSampling2D(2, interpolation='bilinear')(conv_up_3_2)
conc_4 = Concatenate()([conv_1_2, up_4])
conv_up_4_1 = Conv2D(32, (3, 3), padding='same')(conc_4)
conv_up_4_1 = Activation('relu')(conv_up_4_1)

conv_up_4_2 = Conv2D(1, (3, 3), padding='same')(conv_up_4_1)
result = Activation('sigmoid')(conv_up_4_2)


model = Model(inputs=inp, outputs=result)
model.compile(optimizer='adam', loss='binary_crossentropy') 
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ReduceLROnPlateau(patience=3, verbose=1),
    ModelCheckpoint('model-unet.h5', verbose=1, save_best_only=True, save_weights_only=True)
]



In [ ]:
epochs = 60
batch_size = 32
history = model.fit(X_train, Y_train,
                    validation_data=[X_valid, Y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=callbacks, 
                    )

In [ ]:
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.uint8)

print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(train_ids)):
    path = path_test
    img = load_img(path + 'images/' + id_)
    x = img_to_array(img)[:,:,1]
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    X_test[n] = x
    
print('Done!')

In [ ]:
# Load best model
model.load_weights('model-unet.h5')

In [ ]:
model.evaluate(X_valid,  Y_valid, verbose=1)

In [ ]:
# Predict on train, val and test
preds_train = model.predict(X_train,  verbose=1)
preds_val = model.predict( X_valid,  verbose=1)
preds_test = model.predict( X_test,  verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (101, 101), 
                                       mode='constant', preserve_range=True))

In [ ]:
preds_test_upsampled[0].shape

In [ ]:
def plot_sample(X, y, preds):
    ix = random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 3, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='Greys')
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Seismic')
    ax[1].imshow(y[ix].squeeze())
    ax[1].set_title('Salt')

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('Salt Pred');

In [ ]:
plot_sample(X_train, Y_train, preds_train)

In [ ]:
plot_sample(X_valid, y_valid, preds_val)

In [ ]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0
    
    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

pred_dict = {id_[:-4]:RLenc(np.round(preds_test_upsampled[i] > 0.5)) for i,id_ in tqdm_notebook(enumerate(test_ids))}
        
        

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')